In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import *
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Estimator, Options

# This script is for extracting the probability distributions with/without different
# error reduction techniques from IBM hardware. 7 Qubits limits it to max 6 generals.
# These probability distributions are then sampled from in the full scheme.

In [ ]:
# Creating starting states
states = []
for ns in range(3,7):
    state = np.zeros(2**(ns+1))
    state[2**(ns-1)-1] = np.sqrt(1/3)  # Retreat order from commander
    state[2**(ns) + 2**(ns-1)] = np.sqrt(1/3)  # Attack order from commander
    for i in range(ns-1):
        state[2**(ns-1) + 2**i] = np.sqrt(1/(6*ns-6))  # commander '01'
        state[2**(ns) + 2**(ns-1) - 1 - 2**i] = np.sqrt(1/(6*ns-6))  # commander '10'
    states.append(state)

# Creating Circuits
circuits = []
for qbs in range(4,8): # looping over number of qubits
    circuit = QuantumCircuit(qbs)
    circuit.initialize(states[qbs-4], circuit.qubits)
    circuit.measure_all()
    circuits.append(circuit)



In [ ]:
results = []
repeats = 100
service = QiskitRuntimeService()
print('starting session')
with Session(service=service, backend='ibm_nairobi') as session:
    sampler1 = Sampler(session=session, options=Options(optimization_level=0, resilience_level=0))
    sampler2 = Sampler(session=session, options=Options(optimization_level=0, resilience_level=1))
    sampler3 = Sampler(session=session, options=Options(optimization_level=3, resilience_level=0))
    sampler4 = Sampler(session=session, options=Options(optimization_level=3, resilience_level=1))
    print('running jobs')
    job1 = sampler1.run(repeats*circuits, shots=8192).result().quasi_dists
    job2 = sampler2.run(repeats*circuits, shots=8192).result().quasi_dists
    job3 = sampler3.run(repeats*circuits, shots=8192).result().quasi_dists
    job4 = sampler4.run(repeats*circuits, shots=8192).result().quasi_dists
    print('saving results')
    results.append([qd.nearest_probability_distribution() for qd in job1])
    results.append([qd.nearest_probability_distribution() for qd in job2])
    results.append([qd.nearest_probability_distribution() for qd in job3])
    results.append([qd.nearest_probability_distribution() for qd in job4])
    session.close()

# open file in write mode
with open('results.txt', 'w+') as fp:
    for item in results:  # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')


In [ ]:
# Loading results
result = [{0: 0.0494384765625, 1: 0.0205078125, 2: 0.0203857421875, 3: 0.142333984375, 4: 0.018310546875, 5: 0.097900390625, 6: 0.088134765625, 7: 0.01123046875, 8: 0.012451171875, 9: 0.0723876953125, 10: 0.05078125, 11: 0.0177001953125, 12: 0.30615234375, 13: 0.038330078125, 14: 0.0283203125, 15: 0.025634765625}, {0: 0.04207188288628177, 1: -0.0032442393066738203, 2: 0.05749966502187831, 3: 0.27799405527957266, 4: 0.006527856623173984, 5: 0.09272029009353665, 6: 0.05216199368476949, 7: -0.008582007392747568, 8: 0.005183777582416575, 9: 0.17612585673680378, 10: 0.09290837276421766, 11: -0.020263010518109446, 12: 0.20296524930832485, 13: 0.010128072742279916, 14: 0.006801663077957821, 15: 0.009000521416317351}, {0: 0.0384521484375, 1: 0.0152587890625, 2: 0.0322265625, 3: 0.1888427734375, 4: 0.0103759765625, 5: 0.0352783203125, 6: 0.0806884765625, 7: 0.0228271484375, 8: 0.0135498046875, 9: 0.0955810546875, 10: 0.102294921875, 11: 0.0186767578125, 12: 0.2501220703125, 13: 0.0338134765625, 14: 0.0443115234375, 15: 0.0177001953125}, {0: 0.024270114954547, 1: -0.0031743231219098543, 2: 0.022777612953067374, 3: 0.23841674313140127, 4: 0.0024392948887869267, 5: 0.09423544746022561, 6: 0.11278917844665547, 7: -0.005277050588998594, 8: -3.65231702600603e-05, 9: 0.07543261259699877, 10: 0.1136133938981531, 11: -0.006190388746608885, 12: 0.28981531536980737, 13: 0.016142089820538705, 14: 0.0029057818794457667, 15: 0.021840700228150017}, {0: 0.0238037109375, 1: 0.01513671875, 16: 0.016845703125, 17: 0.013427734375, 18: 0.005859375, 19: 0.0223388671875, 20: 0.0072021484375, 21: 0.031005859375, 22: 0.014892578125, 23: 0.010498046875, 24: 0.104736328125, 25: 0.039794921875, 26: 0.0394287109375, 27: 0.027099609375, 28: 0.018798828125, 29: 0.011962890625, 30: 0.01220703125, 31: 0.0220947265625, 2: 0.0272216796875, 3: 0.028076171875, 4: 0.0574951171875, 5: 0.0230712890625, 6: 0.0177001953125, 7: 0.137451171875, 8: 0.0250244140625, 9: 0.0545654296875, 10: 0.049072265625, 11: 0.034423828125, 12: 0.0582275390625, 13: 0.01513671875, 14: 0.021240234375, 15: 0.01416015625}, {0: 0.06613673910378642, 1: 0.021906623894837638, 2: 0.062165482117814315, 3: 0.025989779289223122, 4: 0.051702833837491455, 5: 0.023689525500708766, 6: 0.03160693839763546, 7: 0.10257068294499815, 8: 0.020372519005591238, 9: 0.037866375311295455, 10: 0.04278508145335765, 11: 0.00900482937870487, 12: 0.011019638046704795, 13: 0.006740756177076807, 14: 0.008572787700964401, 15: 0.003731746838720239, 16: 0.037370411824890974, 17: 0.021957801099786376, 18: 0.017683314542871722, 19: 0.054791581127321125, 20: 0.0004747662999892667, 21: 0.05013079869652378, 22: 0.024764103271860335, 23: 0.0004449267519185114, 24: 0.1595135586395603, 25: 0.025864623660679646, 26: 0.0335368932940498, 27: 0.02778844198702431, 28: 3.9184416801703946e-05, 29: 0.0033320956314306436, 30: 0.01262721327010867, 31: 0.0038179464862720903}, {0: 0.0108642578125, 1: 0.0074462890625, 16: 0.026611328125, 17: 0.0107421875, 18: 0.010986328125, 19: 0.027587890625, 20: 0.0089111328125, 21: 0.032470703125, 22: 0.0511474609375, 23: 0.015625, 24: 0.13427734375, 25: 0.0123291015625, 26: 0.047607421875, 27: 0.0216064453125, 28: 0.0335693359375, 29: 0.0181884765625, 30: 0.01220703125, 31: 0.01611328125, 2: 0.019775390625, 3: 0.007080078125, 4: 0.02392578125, 5: 0.03271484375, 6: 0.044921875, 7: 0.1800537109375, 8: 0.015380859375, 9: 0.0350341796875, 10: 0.058837890625, 11: 0.012451171875, 12: 0.0184326171875, 13: 0.0125732421875, 14: 0.019287109375, 15: 0.021240234375}, {0: 0.017459169137396733, 1: 0.0038310317359201594, 2: 0.017715822433806473, 3: 0.006169724762787619, 4: 0.03244485891077184, 5: 0.008644246645232131, 6: 0.024036781139977946, 7: 0.1502806487207469, 8: 0.013734324689648992, 9: 0.05879433248995324, 10: 0.03141949279797691, 11: 0.0043789956188105975, 12: 0.06964505721883224, 13: 0.00747232853815177, 14: 0.02489185136885409, 15: 0.045381451041932065, 16: 0.03484613670458389, 17: 0.005336761920036002, 18: 0.008659871519777644, 19: 0.03824244328298328, 20: 0.007884496034771987, 21: 0.04041843064977109, 22: 0.05086032195779548, 23: 0.00759130307731637, 24: 0.1610651404790703, 25: 0.019886299380451035, 26: 0.018385965583352752, 27: 0.02343132865165182, 28: 0.01598405701714443, 29: 0.016535294795231872, 30: 0.017048590061515646, 31: 0.017523441633746688}, {0: 0.033935546875, 1: 0.02880859375, 16: 0.0216064453125, 17: 0.025390625, 18: 0.018798828125, 19: 0.0255126953125, 20: 0.0164794921875, 21: 0.01318359375, 22: 0.0162353515625, 23: 0.0103759765625, 24: 0.0159912109375, 25: 0.016357421875, 26: 0.0108642578125, 27: 0.01123046875, 28: 0.010498046875, 29: 0.009033203125, 30: 0.0086669921875, 31: 0.0076904296875, 2: 0.022216796875, 32: 0.0145263671875, 33: 0.016845703125, 34: 0.0101318359375, 35: 0.0130615234375, 36: 0.010986328125, 37: 0.01025390625, 38: 0.0076904296875, 39: 0.015380859375, 40: 0.0126953125, 41: 0.0103759765625, 42: 0.0086669921875, 43: 0.0120849609375, 44: 0.0091552734375, 45: 0.0145263671875, 46: 0.0087890625, 47: 0.012939453125, 3: 0.0264892578125, 48: 0.0225830078125, 49: 0.018798828125, 50: 0.01220703125, 51: 0.016845703125, 52: 0.01025390625, 53: 0.009033203125, 54: 0.0067138671875, 55: 0.00634765625, 56: 0.007568359375, 57: 0.01123046875, 58: 0.005126953125, 59: 0.010986328125, 60: 0.005859375, 61: 0.00634765625, 62: 0.0040283203125, 63: 0.0068359375, 4: 0.025634765625, 5: 0.023193359375, 6: 0.0299072265625, 7: 0.018798828125, 8: 0.0159912109375, 9: 0.03466796875, 10: 0.0203857421875, 11: 0.028076171875, 12: 0.0220947265625, 13: 0.0194091796875, 14: 0.0184326171875, 15: 0.045166015625}, {0: 0.03517202650394778, 1: 0.03239685830572886, 2: 0.017553685025145444, 3: 0.015657353218512377, 4: 0.04308133661657733, 5: 0.01893832913592055, 6: 0.021107497772348344, 7: 0.015488035399399755, 8: 0.020581925325675654, 9: 0.03559628886236597, 10: 0.02322160942234329, 11: 0.011213526434928749, 12: 0.020917090140377383, 13: 0.009107591769051522, 14: 0.021360504229637044, 15: 0.038112207442153836, 16: 0.027580084623741608, 17: 0.029575450250704825, 18: 0.013937854796075993, 19: 0.014743225961316374, 20: 0.021598903334516317, 21: 0.013480824290045306, 22: 0.015935410069022157, 23: 0.009716944392915218, 24: 0.018205359960882586, 25: 0.03605963252711983, 26: 0.008036617195742063, 27: 0.00432239049400699, 28: 0.015949952439340664, 29: 0.008203219629958177, 30: 0.005744708955125589, 31: 0.008532247096251444, 32: 0.020120841641041998, 33: 0.01656260290658233, 34: 0.00937139525452508, 35: 0.016868371844029974, 36: 0.014992211912732922, 37: 0.012135179818948424, 38: 0.003494854582976049, 39: 0.011620440867289729, 40: 0.010477795305407472, 41: 0.0121696657883768, 42: 0.006844869228952709, 43: 0.013186223477109226, 44: 0.0028862965478929407, 45: 0.01450470459097162, 46: 0.01046993414889202, 47: 0.007871147562726617, 48: 0.020425300411316117, 49: 0.021555801437211523, 50: 0.012666557194495314, 51: 0.015311690270886932, 52: 0.01515976049392647, 53: 0.011730681310831478, 54: 0.012994447320083216, 55: 0.006931850321680969, 56: 0.009755589291878081, 57: 0.01755006575712203, 58: 0.006656565113269705, 59: 0.007467322649403525, 60: 0.00630496080648398, 61: 0.007816642931088524, 62: 0.006992997177156387, 63: 0.00597454041383077}, {0: 0.0096435546875, 1: 0.0087890625, 16: 0.010498046875, 17: 0.01708984375, 18: 0.0235595703125, 19: 0.01171875, 20: 0.0162353515625, 21: 0.009765625, 22: 0.015625, 23: 0.0145263671875, 24: 0.0172119140625, 25: 0.013671875, 26: 0.0299072265625, 27: 0.015380859375, 28: 0.017822265625, 29: 0.0166015625, 30: 0.018798828125, 31: 0.0157470703125, 2: 0.0096435546875, 32: 0.0152587890625, 33: 0.0120849609375, 34: 0.012451171875, 35: 0.0103759765625, 36: 0.012939453125, 37: 0.0072021484375, 38: 0.0087890625, 39: 0.014404296875, 40: 0.014404296875, 41: 0.0086669921875, 42: 0.011962890625, 43: 0.0220947265625, 44: 0.0079345703125, 45: 0.0133056640625, 46: 0.0225830078125, 47: 0.013916015625, 3: 0.0089111328125, 48: 0.03759765625, 49: 0.01904296875, 50: 0.017578125, 51: 0.0133056640625, 52: 0.0255126953125, 53: 0.021728515625, 54: 0.017578125, 55: 0.0196533203125, 56: 0.014404296875, 57: 0.0152587890625, 58: 0.015625, 59: 0.012451171875, 60: 0.0120849609375, 61: 0.0152587890625, 62: 0.0128173828125, 63: 0.01318359375, 4: 0.0113525390625, 5: 0.00927734375, 6: 0.014404296875, 7: 0.00830078125, 8: 0.0166015625, 9: 0.0133056640625, 10: 0.023193359375, 11: 0.0135498046875, 12: 0.021484375, 13: 0.010498046875, 14: 0.0174560546875, 15: 0.0479736328125}, {0: 0.013871699382460715, 1: 0.006652572659542592, 2: 0.005664090589756375, 3: 0.00724789507135104, 4: 0.01133101642104416, 5: 0.012592413498565472, 6: 0.009591127622725053, 7: 0.010179460598311204, 8: 0.01686852978733632, 9: 0.014669949127492593, 10: 0.024442170347092083, 11: 0.01064542043775633, 12: 0.02273618588089015, 13: 0.015583231528582177, 14: 0.019567660349989198, 15: 0.06853343178321822, 16: 0.006052934389161038, 17: 0.013981010905995813, 18: 0.015339952307558169, 19: 0.012536830806609809, 20: 0.013258980463435772, 21: 0.015491219258828723, 22: 0.010543558388959906, 23: 0.00926335360887297, 24: 0.014298853701660138, 25: 0.010352284441767895, 26: 0.015867598979393015, 27: 0.013187635085042643, 28: 0.03436484558350892, 29: 0.015891240563190006, 30: 0.02081047872138865, 31: 0.01524475584389422, 32: 0.014755569980658474, 33: 0.014560855087229698, 34: 0.011468922782936524, 35: 0.012332476363356845, 36: 0.019786384905316888, 37: 0.009376636185489314, 38: 0.006848961501184192, 39: 0.0172349358376689, 40: 0.008555466221490443, 41: 0.013734277322894275, 42: 0.005914346099412379, 43: 0.022790048100540663, 44: 0.007135859663373328, 45: 0.019906406805753406, 46: 0.021090479626297433, 47: 0.010964764909793022, 48: 0.0485661631366881, 49: 0.016300526595286798, 50: 0.013307496404138367, 51: 0.019022761750192303, 52: 0.025467808811259582, 53: 0.03550170580625301, 54: 0.014628216688059912, 55: 0.018094695301552065, 56: 0.014838925240382501, 57: 0.014282728806447206, 58: 0.008459145335254777, 59: 0.009513476293384936, 60: 0.007646845129616744, 61: 0.013069605234093291, 62: 0.007208499996045223, 63: 0.010972619942567988}, {0: 0.0362548828125, 1: 0.0235595703125, 16: 0.0225830078125, 17: 0.0238037109375, 18: 0.0172119140625, 19: 0.017333984375, 20: 0.01318359375, 21: 0.017578125, 22: 0.011474609375, 23: 0.0123291015625, 24: 0.015869140625, 25: 0.013916015625, 26: 0.0107421875, 27: 0.0113525390625, 28: 0.011474609375, 29: 0.010009765625, 30: 0.007080078125, 31: 0.0101318359375, 2: 0.0206298828125, 32: 0.0133056640625, 33: 0.01025390625, 34: 0.0081787109375, 35: 0.0093994140625, 36: 0.00634765625, 37: 0.0068359375, 38: 0.005126953125, 39: 0.00537109375, 40: 0.0087890625, 41: 0.0062255859375, 42: 0.007080078125, 43: 0.0052490234375, 44: 0.0054931640625, 45: 0.0040283203125, 46: 0.00341796875, 47: 0.004150390625, 3: 0.022216796875, 48: 0.0076904296875, 49: 0.0064697265625, 50: 0.0037841796875, 51: 0.005859375, 52: 0.003173828125, 53: 0.00390625, 54: 0.00341796875, 55: 0.0029296875, 56: 0.0042724609375, 57: 0.00439453125, 58: 0.0047607421875, 59: 0.0030517578125, 60: 0.0028076171875, 61: 0.0029296875, 62: 0.003173828125, 63: 0.0032958984375, 4: 0.0179443359375, 64: 0.01171875, 65: 0.014404296875, 66: 0.0113525390625, 67: 0.0089111328125, 68: 0.0078125, 69: 0.008056640625, 70: 0.0081787109375, 71: 0.008056640625, 72: 0.009033203125, 73: 0.00830078125, 74: 0.005615234375, 75: 0.0067138671875, 76: 0.005615234375, 77: 0.0054931640625, 78: 0.004638671875, 79: 0.0052490234375, 5: 0.0203857421875, 80: 0.0086669921875, 81: 0.009033203125, 82: 0.0064697265625, 83: 0.0050048828125, 84: 0.00439453125, 85: 0.005126953125, 86: 0.00439453125, 87: 0.0054931640625, 88: 0.0057373046875, 89: 0.0052490234375, 90: 0.00390625, 91: 0.00439453125, 92: 0.0048828125, 93: 0.00439453125, 94: 0.0030517578125, 95: 0.004638671875, 6: 0.017333984375, 96: 0.0059814453125, 97: 0.0062255859375, 98: 0.0042724609375, 99: 0.0037841796875, 100: 0.0032958984375, 101: 0.00341796875, 102: 0.00244140625, 103: 0.0025634765625, 104: 0.0029296875, 105: 0.0035400390625, 106: 0.0025634765625, 107: 0.0020751953125, 108: 0.0018310546875, 109: 0.001708984375, 110: 0.001220703125, 111: 0.0023193359375, 7: 0.0166015625, 112: 0.0018310546875, 113: 0.003662109375, 114: 0.0013427734375, 115: 0.0029296875, 116: 0.0015869140625, 117: 0.0020751953125, 118: 0.001953125, 119: 0.001220703125, 120: 0.0023193359375, 121: 0.0020751953125, 122: 0.001708984375, 123: 0.001708984375, 124: 0.001220703125, 125: 0.0009765625, 126: 0.0006103515625, 127: 0.001220703125, 8: 0.0198974609375, 9: 0.0206298828125, 10: 0.01953125, 11: 0.0164794921875, 12: 0.01416015625, 13: 0.0147705078125, 14: 0.0140380859375, 15: 0.01171875}, {0: 0.03673520006407206, 1: 0.031306486997370604, 2: 0.025086682234066404, 3: 0.031183014091899098, 4: 0.025952022065156184, 5: 0.019675797348468575, 6: 0.012846652151324675, 7: 0.02034558851614685, 8: 0.01063755888461174, 9: 0.015666087965561397, 10: 0.009880152070727576, 11: 0.007017445353939576, 12: 0.010997049489704529, 13: 0.009307377361018797, 14: 0.008418841426092056, 15: 0.005869185244411801, 16: 0.019987084445613854, 17: 0.02105261635796426, 18: 0.013814956052139667, 19: 0.020241382292762806, 20: 0.013313662114757752, 21: 0.019346732228616513, 22: 0.014172529499716709, 23: 0.01060396116229114, 24: 0.010275739011234132, 25: 0.008442849409989326, 26: 0.009715233374435245, 27: 0.008054361888973162, 28: 0.007365179152542892, 29: 0.008525862093813313, 30: 0.004300955176741278, 31: 0.008465800515550043, 32: 0.013664576940103125, 33: 0.014083904817754088, 34: 0.006496646556247815, 35: 0.008508231613339284, 36: 0.0058770090750743225, 37: 0.01012132401528615, 38: 0.007621356144256701, 39: 0.008276078391709966, 40: 0.006466415659037293, 41: 0.005544676698771184, 42: 0.005371272913862433, 43: 0.002474560279346745, 44: 0.0041933783610268865, 45: 0.003658080146687111, 46: 0.0018037577148935227, 47: 0.001838801471607556, 48: 0.00756222866606878, 49: 0.009329148245624084, 50: 0.007214294030906364, 51: 0.007411821305311492, 52: 0.005462171063468417, 53: 0.004119978448452123, 54: 0.0046092420504612415, 55: 0.003373028682116016, 56: 0.003199366417502892, 57: 0.005139548468453207, 58: 0.0015189212047438153, 59: 0.0038901129303984274, 60: 0.003732443535100862, 61: 0.0010237368886841125, 62: 0.0005645139384644481, 63: 0.003068188193906222, 64: 0.015361818693572235, 65: 0.01830730129440961, 66: 0.012654721303330705, 67: 0.013310681239360862, 68: 0.010304445203837049, 69: 0.009598063181535737, 70: 0.009659452273935437, 71: 0.006527227206850341, 72: 0.006403307565118028, 73: 0.005973367280694859, 74: 0.0063554104927254825, 75: 0.005665874468835733, 76: 0.0031724445877173483, 77: 0.003449803151321645, 78: 0.0019164165716528353, 79: 0.007464578975853832, 80: 0.01261902085504926, 81: 0.008970958501236852, 82: 0.00719859195561352, 83: 0.008961139602363243, 84: 0.006645546438281595, 85: 0.00616451756613069, 86: 0.004731374893419598, 87: 0.006705867759807306, 88: 0.0031340806592110872, 89: 0.006588904727379916, 90: 0.0029915915801133255, 91: 0.0038714790833369083, 92: 0.00163721504327556, 93: 0.004695004345915182, 94: 0.00118126660820077, 95: 0.002706418986641617, 96: 0.009174331991274582, 97: 0.011215421012331592, 98: 0.005543216712829213, 99: 0.006935860023168904, 100: 0.0050879006706384105, 101: 0.004616520071380569, 102: 0.0024959770029128084, 103: 0.005286196705067007, 104: 0.002604464601397163, 105: 0.004003956004948296, 106: 0.0018612815928140088, 107: 0.003662277013501231, 108: 0.0024303292576886753, 109: 0.0016922856370115995, 110: 0.0015303454926069936, 111: 0.0046610846729672846, 112: 0.0035631321870136317, 113: 0.004403502481073299, 114: 0.005020494468331411, 115: 0.004540951543086759, 116: 0.00419608734947315, 117: 0.004441810938793349, 118: 0.003106083086511498, 119: 0.002331670944705971, 120: 0.003398458737483332, 121: 0.0016771794525655883, 122: 0.0015073723571989528, 123: 0.000918005432331522, 124: 0.0011555215095866468, 125: 0.0029695314264262486, 126: 0.0020907273911629706, 127: 0.0011272692246084455}, {0: 0.005615234375, 1: 0.0064697265625, 16: 0.0067138671875, 17: 0.0086669921875, 18: 0.0062255859375, 19: 0.0072021484375, 20: 0.0089111328125, 21: 0.0125732421875, 22: 0.0091552734375, 23: 0.0096435546875, 24: 0.0069580078125, 25: 0.0069580078125, 26: 0.00537109375, 27: 0.0062255859375, 28: 0.00830078125, 29: 0.009521484375, 30: 0.0069580078125, 31: 0.009033203125, 2: 0.0048828125, 32: 0.0072021484375, 33: 0.012939453125, 34: 0.00732421875, 35: 0.0081787109375, 36: 0.0074462890625, 37: 0.0106201171875, 38: 0.00732421875, 39: 0.0086669921875, 40: 0.00634765625, 41: 0.006591796875, 42: 0.0048828125, 43: 0.0059814453125, 44: 0.0047607421875, 45: 0.006591796875, 46: 0.0069580078125, 47: 0.0064697265625, 3: 0.0068359375, 48: 0.006103515625, 49: 0.0079345703125, 50: 0.0067138671875, 51: 0.0067138671875, 52: 0.00830078125, 53: 0.0096435546875, 54: 0.007080078125, 55: 0.0089111328125, 56: 0.0042724609375, 57: 0.005615234375, 58: 0.005859375, 59: 0.00634765625, 60: 0.006591796875, 61: 0.0078125, 62: 0.0045166015625, 63: 0.005859375, 4: 0.005859375, 64: 0.0074462890625, 65: 0.01025390625, 66: 0.0057373046875, 67: 0.0101318359375, 68: 0.008544921875, 69: 0.009765625, 70: 0.0072021484375, 71: 0.0096435546875, 72: 0.0068359375, 73: 0.0067138671875, 74: 0.0084228515625, 75: 0.0078125, 76: 0.0059814453125, 77: 0.010009765625, 78: 0.0064697265625, 79: 0.006103515625, 5: 0.0089111328125, 80: 0.0068359375, 81: 0.00830078125, 82: 0.0081787109375, 83: 0.0072021484375, 84: 0.0069580078125, 85: 0.00927734375, 86: 0.008056640625, 87: 0.0126953125, 88: 0.0076904296875, 89: 0.008544921875, 90: 0.005859375, 91: 0.00927734375, 92: 0.0086669921875, 93: 0.0106201171875, 94: 0.0098876953125, 95: 0.00927734375, 6: 0.007080078125, 96: 0.0162353515625, 97: 0.0120849609375, 98: 0.010009765625, 99: 0.0120849609375, 100: 0.0086669921875, 101: 0.011962890625, 102: 0.011474609375, 103: 0.0096435546875, 104: 0.009521484375, 105: 0.0084228515625, 106: 0.0096435546875, 107: 0.010009765625, 108: 0.0079345703125, 109: 0.00732421875, 110: 0.0064697265625, 111: 0.008544921875, 7: 0.008056640625, 112: 0.0086669921875, 113: 0.0098876953125, 114: 0.01025390625, 115: 0.0079345703125, 116: 0.0091552734375, 117: 0.0081787109375, 118: 0.0084228515625, 119: 0.008544921875, 120: 0.0068359375, 121: 0.0064697265625, 122: 0.006103515625, 123: 0.00634765625, 124: 0.0059814453125, 125: 0.0074462890625, 126: 0.0067138671875, 127: 0.0076904296875, 8: 0.00439453125, 9: 0.00390625, 10: 0.0050048828125, 11: 0.0057373046875, 12: 0.00634765625, 13: 0.0062255859375, 14: 0.0048828125, 15: 0.0057373046875}, {0: 0.007426065349126219, 1: 0.008970556251525541, 2: 0.005534232315282933, 3: 0.005875924531695203, 4: 0.008569339992064948, 5: 0.010866855061124643, 6: 0.006236992390117455, 7: 0.00801731512388616, 8: 0.006673280816112581, 9: 0.007176090641846772, 10: 0.0070213982448512294, 11: 0.004514154280540731, 12: 0.007211371393486447, 13: 0.007499362431511528, 14: 0.008668046426234, 15: 0.004190515782494092, 16: 0.007228797238005705, 17: 0.009720523714415475, 18: 0.008358601728201898, 19: 0.006603670405852221, 20: 0.0083637212727348, 21: 0.008789692950362698, 22: 0.010258842840218614, 23: 0.008072689730695803, 24: 0.009573799468467028, 25: 0.006868440097253613, 26: 0.007326758655173209, 27: 0.006050460962947613, 28: 0.009596411247009953, 29: 0.006617255553526466, 30: 0.010097320896008696, 31: 0.012250276506472698, 32: 0.009350343165454486, 33: 0.009971492778996179, 34: 0.009402959196131126, 35: 0.007295623589995631, 36: 0.006868784961896074, 37: 0.00865733253465066, 38: 0.005289173330659757, 39: 0.010019920414788327, 40: 0.007486254409401105, 41: 0.005303989862082117, 42: 0.007382595208858964, 43: 0.0065875361861823965, 44: 0.006808026363533885, 45: 0.004973184845367324, 46: 0.007274897312111303, 47: 0.006405851249768826, 48: 0.00868938642019508, 49: 0.005485808916750456, 50: 0.006463327688760373, 51: 0.005624345868943776, 52: 0.004743744249245088, 53: 0.005685050434815541, 54: 0.00895365383564568, 55: 0.004721539447243725, 56: 0.007707886421642402, 57: 0.0075091950163451325, 58: 0.006140481456340647, 59: 0.005027027124424391, 60: 0.006208333311195896, 61: 0.007558551299652766, 62: 0.006971030074168401, 63: 0.005622071173852555, 64: 0.0096203672248074, 65: 0.007470221299552286, 66: 0.007753763864881525, 67: 0.008742613697589283, 68: 0.011213541044562295, 69: 0.006428716261871202, 70: 0.008645729996434908, 71: 0.008168765054349993, 72: 0.007898800558670442, 73: 0.008213880953413256, 74: 0.009171869002364608, 75: 0.009269476439959321, 76: 0.007956428338123194, 77: 0.010815519554876665, 78: 0.00707957719762241, 79: 0.008133974308513895, 80: 0.008082175362905853, 81: 0.00550487157903072, 82: 0.006230472921151114, 83: 0.008334229185797973, 84: 0.007346416409762312, 85: 0.007737142694207704, 86: 0.007153273366812045, 87: 0.00852521638966746, 88: 0.010010048835066428, 89: 0.007186592131243112, 90: 0.006590614433022418, 91: 0.0072637975818125285, 92: 0.006505958547007623, 93: 0.011755885646851856, 94: 0.007724406324838919, 95: 0.007288903409521796, 96: 0.01533526563736679, 97: 0.011033353203693736, 98: 0.009687350387033876, 99: 0.012765800634219207, 100: 0.009132582123390208, 101: 0.010705931182223717, 102: 0.008638445025753153, 103: 0.007084211665085864, 104: 0.013960717579809509, 105: 0.01133731349891918, 106: 0.01053042132605011, 107: 0.007444627058582364, 108: 0.010160882256041523, 109: 0.008517779981252993, 110: 0.010492050702422444, 111: 0.008301339902456623, 112: 0.008750966487292044, 113: 0.005171556613763817, 114: 0.007349657031748187, 115: 0.0034054411016744635, 116: 0.0056440186124728795, 117: 0.006923984318535702, 118: 0.005799619307132969, 119: 0.0066683890526498596, 120: 0.007140246002789608, 121: 0.007169831824417352, 122: 0.006682354172496466, 123: 0.008051931027157334, 124: 0.006620259948651196, 125: 0.0053878583653251926, 126: 0.005708271755796314, 127: 0.006156130149155676}]

# Reformatting
new_result = []
i = 16
for qpd in result:
    res_tmp = []
    for k,v in qpd.items():
        res_tmp.append((format(k, '0'+str(i//4)+'b'), v**2))
    new_result.append(res_tmp)
    i += 1

# New format as 16 sublists, each containining tuples with binary string and probability
# There are some negative values (not sure why)

In [ ]:
# In order to use np.random.choice I will need to reformat
options, pvals = [], []
for tup in new_result[4]:
    options.append(tup[0])
    pvals.append(tup[1])

tot = np.sum(pvals)
pvals = [val/tot for val in pvals]

test = np.random.choice(options, 1000, p=pvals)

plt.hist(test)
plt.show()

In [ ]:
# I want to get the quasi-prob distribution for 3 generals, up to 6 (for now),
# with and without error mitigation.
# This can be achieved by altering the Optimization level, and Resilience level.
# Optimization level=3 is dynamical decoupling.
# Resilience level=1 is T-REx error mitigation.
# This should mean I need 16 quasi-prob distributions.
# service = QiskitRuntimeService()
# service.backends() # 'ibm_nairobi',  'ibmq_qasm_simulator'